In [ ]:
from pathlib import Path
import sys

THIS_DIR = Path.cwd().resolve()
PROJECT_ROOT = THIS_DIR.parents[1]  # -> ptta/
print(PROJECT_ROOT)

sys.path.insert(0, str(PROJECT_ROOT))

In [2]:
import sys
from pathlib import Path

# 현재 폴더: ptta/other_method/DUA/
# ptta 바로 위의 디렉토리를 sys.path에 추가
PROJECT_PARENT = Path.cwd().parents[1]  # -> ptta/ 의 부모 디렉토리
sys.path.insert(0, str(PROJECT_PARENT))

from os import path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTClearDatasetForObjectDetection, SHIFTCorruptedDatasetForObjectDetection, SHIFTDiscreteSubsetForObjectDetection
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

# import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

## Check GPU Availability

In [3]:
!nvidia-smi

Wed Aug 27 14:04:39 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   41C    P0              34W / 250W |   1922MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 1
ADDITIONAL_GPU = 0
DATA_TYPE = torch.bfloat16

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

INFO: Using device - cuda:1


In [5]:
PROJECT_NAME = "DUA test"
RUN_NAME = "RT-DETR_R50_DUA"

## Dataset

In [6]:
DATA_ROOT = path.normpath(path.join(Path.cwd(), "..", "..", "data"))
print(DATA_ROOT)
dataset = DatasetHolder(
    train=SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, train=True),
    valid=SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, valid=True),
    test=SHIFTCorruptedDatasetForObjectDetection(root=DATA_ROOT, valid=True)
)
DATA_ROOT

[08/27/2025 14:04:40] SHIFT DevKit - INFO - Base: /workspace/ptta/data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f01c9a4b8d0>
[08/27/2025 14:04:40] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' ...


/workspace/ptta/data
INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[08/27/2025 14:04:41] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' Done.
[08/27/2025 14:04:50] SHIFT DevKit - INFO - Loading annotation takes 10.64 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0016-1b62']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -7.53     219.91
boxes2d              torch.Size([1, 26, 4])                    5.00     974.00
boxes2d_classes      torch.Size([1, 26])                       0.00       3.00
boxes2d_track_ids    torch.Size([1, 26])                       0.00      25.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00



[08/27/2025 14:04:55] SHIFT DevKit - INFO - Base: /workspace/ptta/data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f01c9a4b8d0>
[08/27/2025 14:04:55] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' ...
[08/27/2025 14:04:55] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' Done.


...

[08/27/2025 14:04:57] SHIFT DevKit - INFO - Base: /workspace/ptta/data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f01c9a4b8d0>
[08/27/2025 14:04:57] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' ...


Video name: 0116-4859
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[08/27/2025 14:04:59] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' Done.
[08/27/2025 14:05:08] SHIFT DevKit - INFO - Loading annotation takes 10.69 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['007b-4e72']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                  -311.22     226.46
boxes2d              torch.Size([1, 3, 4])                   233.00     802.00
boxes2d_classes      torch.Size([1, 3])                        0.00       1.00
boxes2d_track_ids    torch.Size([1, 3])                        0.00       2.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00

Video name: 007b-4e72
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

'/workspace/ptta/data'

In [7]:
from typing import Iterable, List

def task_to_subset_types(task: str):
    T = SHIFTDiscreteSubsetForObjectDetection.SubsetType

    # weather
    if task == "cloudy":
        return T.CLOUDY_DAYTIME
    if task == "overcast":
        return T.OVERCAST_DAYTIME
    if task == "rainy":
        return T.RAINY_DAYTIME
    if task == "foggy":
        return T.FOGGY_DAYTIME

    # time
    if task == "night":
        return T.CLEAR_NIGHT
    if task in {"dawn", "dawn/dusk"}:
        return T.CLEAR_DAWN
    if task == "clear":
        return T.CLEAR_DAYTIME
    
    # simple
    if task == "normal":
        return T.NORMAL
    if task == "corrupted":
        return T.CORRUPTED

    raise ValueError(f"Unknown task: {task}")

In [8]:
from typing import Optional, Callable

class SHIFTCorruptedTaskDatasetForObjectDetection(SHIFTDiscreteSubsetForObjectDetection):
    def __init__(
            self, root: str, force_download: bool = False,
            train: bool = True, valid: bool = False,
            transform: Optional[Callable] = None, task: str = "clear", target_transform: Optional[Callable] = None
    ):
        super().__init__(
            root=root, force_download=force_download,
            train=train, valid=valid, subset_type=task_to_subset_types(task),
            transform=transform, target_transform=target_transform
        )

In [9]:
# Set Batch Size
BATCH_SIZE = 2, 8, 8, 8  # 4070 Ti
BATCH_SIZE = 32, 64, 64, 32  # A6000

# Dataset Configs
CLASSES = dataset.train.classes
NUM_CLASSES = len(CLASSES)

print(f"INFO: Set batch size - Train: {BATCH_SIZE[0]}, Valid: {BATCH_SIZE[1]}, Test: {BATCH_SIZE[2]}")
print(f"INFO: Number of classes - {NUM_CLASSES} {CLASSES}")

INFO: Set batch size - Train: 32, Valid: 64, Test: 64
INFO: Number of classes - 6 ['pedestrian', 'car', 'truck', 'bus', 'motorcycle', 'bicycle']


In [10]:
class DatasetAdapterForTransformers(BaseDataset):
    def __init__(self, original_dataset, camera='front'):
        self.dataset = original_dataset
        self.camera = camera

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx][self.camera]
        image = item['images'].squeeze(0)

        # Convert to COCO_Detection Format
        annotations = []
        target = dict(image_id=idx, annotations=annotations)
        for box, cls in zip(item['boxes2d'], item['boxes2d_classes']):
            x1, y1, x2, y2 = box.tolist()  # from Pascal VOC format (x1, y1, x2, y2)
            width, height = x2 - x1, y2 - y1
            annotations.append(dict(
                bbox=[x1, y1, width, height],  # to COCO format: [x, y, width, height]
                category_id=cls.item(),
                area=width * height,
                iscrowd=0
            ))

        # Following prepare_coco_detection_annotation's expected format
        # RT-DETR ImageProcessor converts the COCO bbox to center format (cx, cy, w, h) during preprocessing
        # But, eventually re-converts the bbox to Pascal VOC (x1, y1, x2, y2) format after post-processing
        return dict(image=image, target=target)

In [11]:
def collate_fn(batch, preprocessor=None):
    images = [item['image'] for item in batch]
    if preprocessor is not None:
        target = [item['target'] for item in batch]
        return preprocessor(images=images, annotations=target, return_tensors="pt")
    else:
        # If no preprocessor is provided, just assume images are already in tensor format
        return dict(
            pixel_values=dict(pixel_values=torch.stack(images)),
            labels=[dict(
                class_labels=item['boxes2d_classes'].long(),
                boxes=item["boxes2d"].float()
            ) for item in batch]
        )

## Define Model

In [12]:
from transformers import RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig
from transformers.image_utils import AnnotationFormat
from safetensors.torch import load_file

In [13]:
IMG_SIZE = 800

In [14]:
reference_model_id = "PekingU/rtdetr_r50vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = 6

# Set the image size and preprocessor size
reference_config.image_size = 800

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": IMG_SIZE, "width": IMG_SIZE}
reference_preprocessor.do_resize = False

In [15]:
model_pretrained = RTDetrForObjectDetection(config=reference_config)
model_states = load_file("/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors", device="cpu")
model_pretrained.load_state_dict(model_states, strict=False)

for param in model_pretrained.parameters():
    param.requires_grad = False  # Freeze

# Initialize Model
model_pretrained.to(device)

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(

## NORM

In [16]:
from functools import partial
dataloader_discrete = DataLoader(DatasetAdapterForTransformers(dataset.test), batch_size=4, collate_fn=partial(collate_fn, preprocessor=reference_preprocessor))

In [17]:
class LabelDataset(BaseDataset):
    def __init__(self, original_dataset, camera='front'):
        self.dataset = original_dataset
        self.camera = camera

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx][self.camera]
        return item['boxes2d'], item['boxes2d_classes']

In [18]:
def naive_collate_fn(batch):
    return batch

In [19]:
from functools import partial

def test(model, task, raw_data, dataloader_discrete):
    targets = []
    predictions = []

    for idx, lables, inputs in zip(tqdm(range(len(raw_data))), raw_data, dataloader_discrete):
        sizes = [label['orig_size'].cpu().tolist() for label in inputs['labels']]

        with torch.no_grad():
            outputs = model(pixel_values=inputs['pixel_values'].to(device))

        results = reference_preprocessor.post_process_object_detection(
            outputs, target_sizes=sizes, threshold=0.0
        )

        detections = [Detections.from_transformers(results[i]) for i in range(len(results))]
        annotations = [Detections(
            xyxy=lables[i][0].cpu().numpy(),
            class_id=lables[i][1].cpu().numpy(),
        ) for i in range(len(lables))]

        targets.extend(annotations)
        predictions.extend(detections)
    
    mean_average_precision = MeanAveragePrecision().update(
    predictions=predictions,
    targets=targets,
    ).compute()
    per_class_map = {
        f"{CLASSES[idx]}_mAP@0.95": mean_average_precision.ap_per_class[idx].mean()
        for idx in mean_average_precision.matched_classes
    }

    print(f"mAP@0.95_{task}: {mean_average_precision.map50_95:.3f}")
    print(f"mAP50_{task}: {mean_average_precision.map50:.3f}")
    print(f"mAP75_{task}: {mean_average_precision.map75:.3f}")
    for key, value in per_class_map.items():
        print(f"{key}_{task}: {value:.3f}")
    
    return {"mAP@0.95" : mean_average_precision.map50_95,
            "mAP50" : mean_average_precision.map50,
            "mAP75" : mean_average_precision.map75,
            "per_class_mAP@0.95" : per_class_map
            }

In [20]:
from collections import defaultdict

def agg_per_class(dicts):
    """dicts: per_class_map(dict)의 리스트. 예: [{"car_mAP@0.95":0.41, ...}, {...}]"""
    sums = defaultdict(float)
    counts = defaultdict(int)
    for d in dicts:
        for cls, val in d.items():
            sums[cls]  += float(val)
            counts[cls] += 1
    means = {cls: (sums[cls] / counts[cls]) for cls in sums}
    return means


def aggregate_runs(results_list):
    overall_sum = {"mAP@0.95": 0.0, "mAP50": 0.0, "mAP75": 0.0}
    n = len(results_list)

    per_class_maps = []

    for r in results_list:
        overall_sum["mAP@0.95"] += float(r["mAP@0.95"])
        overall_sum["mAP50"]    += float(r["mAP50"])

        overall_sum["mAP75"] += float(r["mAP75"])

        class_mAP = r["per_class_mAP@0.95"]
        per_class_means = agg_per_class([class_mAP])

    overall_mean = {k: (overall_sum[k] / n if n > 0 else 0.0) for k in overall_sum}

    return {
        "overall_sum": overall_sum,            # {"mAP@0.95": ..., "mAP50": ..., "map75": ...}
        "overall_mean": overall_mean,          # 위의 평균          # {"car_mAP@0.95": 합, ...}
        "per_class_mean@0.95": per_class_means,        # {"car_mAP@0.95": 평균, ...}
    }

def print_results(result):
    om = result["overall_mean"]
    print(f"mAP@0.95: {float(om['mAP@0.95']):.3f}")
    print(f"mAP50: {float(om['mAP50']):.3f}")
    print(f"mAP75: {float(om['mAP75']):.3f}")

    for k, v in result["per_class_mean@0.95"].items():
        print(f"{k}: {v:.3f}")

In [21]:
class EMABatchNorm(nn.Module):
    @staticmethod
    def reset_stats(module):
        module.reset_running_stats()
        module.momentum = None
        return module

    @staticmethod
    def find_bns(parent):
        replace_mods = []
        if parent is None:
            return []
        for name, child in parent.named_children():
            child.requires_grad_(False)
            if isinstance(child, nn.BatchNorm2d):
                module = EMABatchNorm.reset_stats(child)
                module = EMABatchNorm(module)
                replace_mods.append((parent, name, module))
            else:
                replace_mods.extend(EMABatchNorm.find_bns(child))

        return replace_mods

    @staticmethod
    def adapt_model(model):
        replace_mods = EMABatchNorm.find_bns(model)
        print(f"| Found {len(replace_mods)} modules to be replaced.")
        for parent, name, child in replace_mods:
            setattr(parent, name, child)
        return model

    def __init__(self, layer):
        super().__init__()
        self.layer = layer

    def forward(self, x):
        # store statistics, but discard result
        self.layer.train()
        self.layer(x)
        # store statistics, use the stored stats
        self.layer.eval()
        return self.layer(x)

In [22]:
def set_bn_momentum(model, momentum: float | None):
    # momentum=None 이면 CMA(누적 평균), 수치(0<α≤1)이면 EMA
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.momentum = momentum

In [23]:
import os, logging
from datetime import datetime
from tqdm.auto import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm  # tqdm → logging 연동

def setup_logger(save_dir, name="dua", level=logging.INFO, mirror_to_stdout=True):
    os.makedirs(save_dir, exist_ok=True)
    log_path = os.path.join(save_dir, f"{name}_{datetime.now():%Y%m%d_%H%M%S}.log")

    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.handlers.clear()

    fmt = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s")
    # 파일 핸들러(전체 기록)
    fh = logging.FileHandler(log_path, encoding="utf-8")
    fh.setFormatter(fmt)
    logger.addHandler(fh)

    # 화면 핸들러(미러링)
    if mirror_to_stdout:
        sh = logging.StreamHandler()
        sh.setFormatter(fmt)
        logger.addHandler(sh)

    logger.propagate = False
    return logger, log_path

In [24]:
import io, contextlib
from tqdm.contrib.logging import logging_redirect_tqdm  # tqdm도 로깅에 정리되도록

class LoggerWriter(io.TextIOBase):
    def __init__(self, logger, level):
        self.logger = logger
        self.level = level
        self._buf = ""
    def write(self, msg):
        # 줄 단위로 로깅(개행/부분문자 처리)
        self._buf += msg
        while "\n" in self._buf:
            line, self._buf = self._buf.split("\n", 1)
            if line.strip():
                self.level(line)
    def flush(self):
        if self._buf.strip():
            self.level(self._buf.strip())
            self._buf = ""

In [ ]:
from transformers.models.rt_detr.modeling_rt_detr import RTDetrFrozenBatchNorm2d
from torchvision import transforms
import os
import copy

def NORM(model, MOMENTOM, save_dir = './norm_checkpoints'):
    """
    model is a pre-trained model.
    """
    logger, log_path = setup_logger(save_dir, name="norm", mirror_to_stdout=True)
    logger.info(f"Log file: {log_path}")
    
    with contextlib.redirect_stdout(LoggerWriter(logger, logger.info)), \
        contextlib.redirect_stderr(LoggerWriter(logger, logger.error)), \
        logging_redirect_tqdm():  # tqdm 진행바도 깔끔히 로그에 남김

        os.makedirs(save_dir, exist_ok=True)

        all_results = []
        
        device = next(model.parameters()).device
        carry_state = copy.deepcopy(model.state_dict())

        for task in ["rainy", "dawn", "night", "clear"]: # "cloudy", "overcast", "foggy",
            logger.info(f"start {task}")
            model.load_state_dict(carry_state)
            best_state = copy.deepcopy(model.state_dict())
            best_mAP95 = -1.0
            no_imp_streak = 0
            PATIENCE = 10
            EVAL_EVERY = 10

            dataloader_discrete = DataLoader(DatasetAdapterForTransformers(SHIFTCorruptedTaskDatasetForObjectDetection(root=DATA_ROOT, train=True, valid=False, task=task)), batch_size=1, collate_fn=partial(collate_fn, preprocessor=reference_preprocessor))

            # dataset_tta_valid
            dataset_valid = SHIFTCorruptedTaskDatasetForObjectDetection(root=DATA_ROOT, valid=True, task=task)
            
            raw_data = DataLoader(LabelDataset(dataset_valid), batch_size=16, collate_fn=naive_collate_fn)
            dataloader_discrete_valid = DataLoader(DatasetAdapterForTransformers(dataset_valid), batch_size=16, collate_fn=partial(collate_fn, preprocessor=reference_preprocessor))

            for batch_i, input in enumerate(tqdm(dataloader_discrete)):
                model.eval()
                for module in model.modules():
                    if isinstance(module, torch.nn.BatchNorm2d):
                        module.momentum = MOMENTOM
                        module.train()
                img = input['pixel_values'].to(device, non_blocking=True)

                _ = model(img)
                model.eval()

                if batch_i % EVAL_EVERY == 0:
                    current_result = test(model, task, raw_data, dataloader_discrete_valid)
                    current_mAP95 = current_result.get("mAP@0.95", current_result.get("mAP@0.95", -1.0))

                    improved = current_mAP95 >= best_mAP95

                    if improved:
                        logger.info(f"[{task}] batch {batch_i}: mAP@0.95 {best_mAP95:.4f} -> {current_mAP95:.4f} ✔")
                        best_mAP95 = current_mAP95
                        best_state = copy.deepcopy(model.state_dict())
                        no_imp_streak = 0
                    else:
                        no_imp_streak += 1
                        logger.info(f"[{task}] batch {batch_i}: mAP@0.95 {current_mAP95:.4f} (no-imp {no_imp_streak}/{PATIENCE})")
                        if no_imp_streak >= PATIENCE:
                            logger.info(f"[{task}] Early stop at batch {batch_i} (no improvement {PATIENCE} times).")
                            break
            model.load_state_dict(best_state)
            with torch.inference_mode():
                final_result = test(model, task, raw_data, dataloader_discrete_valid)

            save_path = os.path.join(save_dir, f"model_{task}.pth")
            torch.save(model.state_dict(), save_path)
            logger.info(f"Saved adapted model after task [{task}] → {save_path}")

            carry_state = copy.deepcopy(model.state_dict())

            all_results.append(final_result)
        
        each_task_mAP_list = aggregate_runs(all_results)

        print_results(each_task_mAP_list)    

In [ ]:
# 
NORM(model_pretrained, MOMENTOM=0.2)

[2025-08-27 14:05:15,434] INFO: Log file: ./norm_checkpoints/norm_20250827_140515.log
[2025-08-27 14:05:15,539] INFO: start cloudy
[2025-08-27 14:05:15,685] INFO: INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
[2025-08-27 14:05:15,687] INFO: INFO: Dataset archive found in the root directory. Skipping download.
[2025-08-27 14:05:15,688] INFO: INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
[2025-08-27 14:05:15,689] INFO: INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
[2025-08-27 14:05:15,690] INFO: INFO: Dataset archive found in the root directory. Skipping download.
[08/27/2025 14:05:15] SHIFT DevKit - INFO - Base: /workspace/ptta/data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f01c9a4b8d0>
[08/27/2025 14:05:15] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/cloudy_daytime/discrete/images/train

  0%|          | 0/13850 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

[2025-08-27 14:09:59,237] INFO: mAP@0.95_cloudy: 0.489
[2025-08-27 14:09:59,240] INFO: mAP50_cloudy: 0.663
[2025-08-27 14:09:59,241] INFO: mAP75_cloudy: 0.535
[2025-08-27 14:09:59,243] INFO: pedestrian_mAP@0.95_cloudy: 0.462
[2025-08-27 14:09:59,244] INFO: car_mAP@0.95_cloudy: 0.562
[2025-08-27 14:09:59,244] INFO: truck_mAP@0.95_cloudy: 0.557
[2025-08-27 14:09:59,245] INFO: bus_mAP@0.95_cloudy: 0.463
[2025-08-27 14:09:59,246] INFO: motorcycle_mAP@0.95_cloudy: 0.446
[2025-08-27 14:09:59,247] INFO: bicycle_mAP@0.95_cloudy: 0.445
[2025-08-27 14:09:59,304] INFO: [cloudy] batch 0: mAP50 -1.0000 -> 0.6628 ✔


  0%|          | 0/150 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

[2025-08-27 20:31:51,918] INFO: mAP@0.95_rainy: 0.449
[2025-08-27 20:31:51,922] INFO: mAP50_rainy: 0.600
[2025-08-27 20:31:51,923] INFO: mAP75_rainy: 0.486
[2025-08-27 20:31:51,924] INFO: pedestrian_mAP@0.95_rainy: 0.434
[2025-08-27 20:31:51,925] INFO: car_mAP@0.95_rainy: 0.523
[2025-08-27 20:31:51,926] INFO: truck_mAP@0.95_rainy: 0.561
[2025-08-27 20:31:51,927] INFO: bus_mAP@0.95_rainy: 0.323
[2025-08-27 20:31:51,928] INFO: motorcycle_mAP@0.95_rainy: 0.437
[2025-08-27 20:31:51,929] INFO: bicycle_mAP@0.95_rainy: 0.418
[2025-08-27 20:31:51,961] INFO: [rainy] batch 250: mAP50 0.6004 (no-imp 6/10)


  0%|          | 0/200 [00:00<?, ?it/s]

[2025-08-27 20:38:37,619] INFO: mAP@0.95_rainy: 0.454
[2025-08-27 20:38:37,622] INFO: mAP50_rainy: 0.608
[2025-08-27 20:38:37,624] INFO: mAP75_rainy: 0.489
[2025-08-27 20:38:37,625] INFO: pedestrian_mAP@0.95_rainy: 0.438
[2025-08-27 20:38:37,626] INFO: car_mAP@0.95_rainy: 0.530
[2025-08-27 20:38:37,627] INFO: truck_mAP@0.95_rainy: 0.571
[2025-08-27 20:38:37,628] INFO: bus_mAP@0.95_rainy: 0.333
[2025-08-27 20:38:37,629] INFO: motorcycle_mAP@0.95_rainy: 0.435
[2025-08-27 20:38:37,630] INFO: bicycle_mAP@0.95_rainy: 0.416
[2025-08-27 20:38:37,661] INFO: [rainy] batch 260: mAP50 0.6078 (no-imp 7/10)


  0%|          | 0/200 [00:00<?, ?it/s]